In [ ]:
import cupy as cp
import numpy as np

user - время, затраченное на сами вычисления

sys - время, затраченное всеми процессорами на выполнение связанных с системой задач, таких как выделение памяти, например.

In [ ]:
np_array = np.random.random_sample(5000000)
cp_array = cp.random.random_sample(5000000)

In [ ]:
%time np_array.mean()

In [ ]:
%time np_array.sum()

In [ ]:
%time cp_array.mean()

In [ ]:
%time cp_array.sum()

In [ ]:
def logistic_numpy(y):
    return 1 / (1 + np.exp(-y))

def logistic_cupy(y):
    return 1 / (1 + cp.exp(-y))

In [ ]:
%time logistic_numpy(np_array)

In [ ]:
%time logistic_cupy(cp_array)

А можно ли написать такую функцию, чтобы она при наличии numpy массива использовала CPU, а при наличии cupy GPU?

In [ ]:
def univers_logistic(y):
    yp = cp.get_array_module(y)
    return 1 / (1 + yp.exp(y)) 

def universe_dot(x, y):
    xp = cp.get_array_module(y)
    return xp.dot(x, y)

In [ ]:
%time univers_logistic(np_array)

In [ ]:
%time univers_logistic(cp_array)

# Как переключать видеокарты?

Глобальное переключение

In [ ]:
cp.cuda.Device(0).use()

Переключение в рамках контекстного менеджера

In [ ]:
with cp.cuda.Device(0):
    gpu_0 = cp.array([1, 2, 3, 4, 5])
with cp.cuda.Device(1):
    gpu_1 = cp.array([1, 2, 3, 4, 5])
    
print(f'gpu_0 device - {gpu_0.device}, gpu_1 device - {gpu_1.device}')

Проводить операции с массивами на разных видеокартах напрямую нельзя

In [ ]:
gpu_0 + gpu_1

In [ ]:
gpu_1.device

In [ ]:
with cp.cuda.Device(0):
    gpu_1 = cp.asarray(gpu_1)
print(f'gpu_1 device - {gpu_1.device}')
gpu_0 + gpu_1

# Передать массив между CPU и GPU

In [ ]:
#первый способ
x_cpu = cp.asnumpy(gpu_0)

In [ ]:
x_cpu

In [ ]:
#второй способ
x_cpu = gpu_0.get()

In [ ]:
x_cpu

# Собственные kernel

Тип можно указать явно, но если все типы одинаковы и могут меняться, то указывается тип T, который будет определен компилятором

In [ ]:
cupy.ElementwiseKernel(in_params, # входные параметры
                       out_params, # выходные параметры
                       operation, # тело цикла, напианное на CUDA-C/C++
                       name=u'kernel', # имя функции ядра
                       reduce_dims=True, # если False, то формы массивов сохраняются. Иначе размерности уменьшаются до минимума
                                         # и это позволяет ускорять функции
                       preamble=u'', # фрагмент кода CUDA-C / C++, вставляемый в верхнюю часть файла cu
                       no_return=False, # метод __call__ ничего не возвращает если True
                       return_tuple=False, # если True, то всегда возвращается tuple, даже если 1 елемент
                       **kwargs)

In [ ]:
squared_diff = cp.ElementwiseKernel(
    in_params='float32 x, float32 y',
    out_params='float32 z',
    operation='z = (x - y) * (x - y)',
    name='squared_diff')

In [ ]:
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
squared_diff(x, 5)

In [ ]:
#Можно передать массив, в который записывается результат
z = cp.empty((2, 5), dtype=np.float32)
squared_diff(x, 5, z)

In [ ]:
cupy.ReductionKernel(in_params, # входные параметры
                     out_params, # выходные параметры
                     map_expr, # map операция для входных значений
                     reduce_expr, # reduce операция для входных значений
                     post_map_expr, # map операция для взначений после reduce операции
                     identity, #значение идентификатора для начала reduce операций
                     name=u'reduce_kernel',
                     reduce_type=None,
                     reduce_dims=True,
                     preamble=u'',
                     options=() # дополнительные опции для компиляции
                    )

In [ ]:
l2norm_kernel = cp.ReductionKernel(
    'T x',  # input params
    'T y',  # output params
    'x * x',  # map
    'a + b',  # reduce
    'y = sqrt(a)',  # post-reduction map
    '0',  # identity value
    'l2norm'  # kernel name
)
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
l2norm_kernel(x, axis=1)

cp.RawKernel для кода на CUDA source code

ElementwiseKernel и ReductionKernel могут быть написаны проще через декоратор @cp.fuse()

In [ ]:
@cp.fuse()
def squared_diff_dec(x, y):
    return (x - y) * (x - y)

squared_diff_dec(x, 5)

In [ ]:
@cp.fuse()
def l2norm_dec(x):
    return cp.sqrt(cp.sum(cp.power(x, 2), axis=1))

l2norm_dec(x)

In [ ]:
%timeit squared_diff(x, 5)

Помним, что данные уже на GPU и это сравнение с cupy без оптимизации!

In [ ]:
%timeit (x - 5) * (x - 5)

In [ ]:
%timeit squared_diff_dec(x, 5)

In [ ]:
%timeit l2norm_kernel(x, axis=1)

Помним, что данные уже на GPU и это сравнение с cupy без оптимизации!

In [ ]:
%timeit cp.sqrt(cp.sum(cp.power(x, 2), axis=1))

In [ ]:
%timeit l2norm_dec(x)

# Домашнее задание

Так как синтаксис очень похож на numpy, то ничего сложного в cupy нет. 
Придумайте какую-нибудь функцию только с использованием numpy и не в одну операцию, чтобы она несла какой-то смылс. Что-то аналогичное функции из занятия l2norm_dec:

    def l2norm_dec(x):
        return cp.sqrt(cp.sum(cp.power(x, 2), axis=1))
        
Запишите эту же функцию при помощи cupy

Третья функция - cupy + декоратор @cp.fuse()

Сравните время выполнения=)